In [ ]:
!pip install xgboost

In [ ]:

import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

In [ ]:
# Import the data
ppi_clean = pd.read_csv('/kaggle/input/dataset-for-ml-project/ppi_clean.csv')
ppi_clean

In [ ]:
# Remove columns that shouldn't be features
def remove_non_features(df: pd.DataFrame, non_feature_cols: list[str]) -> pd.DataFrame:
    return df.drop(non_feature_cols, axis=1)

In [ ]:
# Split data into train/validation and test set, taking into account the protein groups
from sklearn.model_selection import GroupShuffleSplit

def split_data_by_group(X: pd.DataFrame, y: pd.Series, groups: pd.Series, **kwargs) -> tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    gss = GroupShuffleSplit(**kwargs)

    for train_idx, test_idx in gss.split(X, y, groups):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

    assert set(X_train.uniprot_id).intersection(set(X_test.uniprot_id)) == set(), 'The same uniprot ID appears in both the train and test set!'
    return X_train, X_test, y_train, y_test

def select_window_size(df: pd.DataFrame, window_size) -> pd.DataFrame:
    '''Remove the window columns that are not used, i.e. not specified in window_size'''
    keep_cols = set()
    wm_cols = df.columns[df.columns.str.match(r"^\d+_wm")].tolist()
    all_cols = set(wm_cols)

        # Regex for [number]_wm
      #pssm_cols = [col for col in df.columns if col.startswith('pssm') or col.startswith('prob')]
      #extra_cols = ['rel_surf_acc', 'normalized_abs_surf_acc', 'normalized_hydropathy_index']
      

    keep_cols.update(set([col for col in df.columns if col.startswith(f"{window_size}_wm")]))



    cols_to_remove = list(set(all_cols).difference(set(keep_cols)))
    cols_before = frozenset(df.columns)
    df = df.drop(cols_to_remove, axis=1)
    cols_after = frozenset(df.columns)
    
    return df




In [ ]:
from itertools import combinations
from sklearn.model_selection import GridSearchCV


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GroupKFold
from sklearn.base import BaseEstimator
from sklearn.metrics import roc_auc_score, recall_score

def kfold_cv_by_group(X: pd.DataFrame, y: pd.Series, groups: pd.Series, model: BaseEstimator, **kwargs) -> pd.DataFrame:
    group_kfold = GroupKFold(**kwargs)

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
y = ppi_clean.p_interface
X_non_windowed = ppi_clean.drop('p_interface', axis=1)
non_feature_cols = ['domain', 'aa_ProtPosition', 'uniprot_id']
X_non_windowed = remove_non_features(X_non_windowed, non_feature_cols)
groups = ppi_clean.uniprot_id
X = select_window_size(X_non_windowed, [9])

In [ ]:



for i in [0,1,2]:
    
    
    if i == 0:
        gs_model =  xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight = 7.4)
        gs_params = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 6, 9],  
    'learning_rate': [0.05],  
    'subsample': [0.8],  
    'colsample_bytree': [0.8],  
    'gamma': [0.1],  
    'reg_alpha': [0, 0.01, 0.1],  
    'reg_lambda': [1, 3, 5] 
}

    if i == 1:
        gs_model = RandomForestClassifier(class_weight='balanced', n_estimators = 100)
        gs_params = {
    'max_depth': [20, 25, 30],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [16, 32, 64],  
    'max_features': ['sqrt']
        }
    else:
        gs_model = DecisionTreeClassifier(class_weight = 'balanced')
        gs_params = {
        'criterion': ['gini', 'entropy'],  
    'max_depth': [10, 20, 30, 50],  
    'min_samples_split': [2, 5, 10, 20],  
    'min_samples_leaf': [1, 2, 5, 10], 
    'max_features': ['sqrt', 'log2', None], 
    'max_leaf_nodes': [None, 10, 20, 30], 
        }
    
        
    
        
    gridsearch = GridSearchCV(gs_model, param_grid=gs_params, n_jobs=-1, scoring='roc_auc', cv=GroupKFold(n_splits=5))
    
   
    gridsearch.fit(X, y, groups=groups)
    print(gridsearch.best_params_, gridsearch.best_score_)









    X = select_window_size(X_non_windowed, list(selection))
    gridsearch.fit(X, y, groups=groups)

    list_results.append([size, selection, gridsearch.best_params_, gridsearch.best_score_])
    results_df = pd.DataFrame(list_results)
    results_df.to_csv('/kaggle/working/apadfasza.csv')






In [ ]:
results_df.to_csv('/kaggle/working/my_data.csv', index=False)